In [1]:
import Pkg
Pkg.add("CSV")
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("DataStructures")
Pkg.add("TextAnalysis")
using DataFrames
using CSV
using TextAnalysis, DataStructures
import Dates

In [2]:
startTime = Dates.Time(Dates.now()) 

13:36:29.918

In [3]:
df = DataFrame(CSV.File("/home/joelkik/HPC/Project/Data/amazon_reviews_us_Apparel_v1_00.tsv"))

┌ Warning: thread = 4 warning: only found 14 / 15 columns around data row: 3678077. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 5 warning: only found 14 / 15 columns around data row: 5076232. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 3 warning: parsed expected 15 columns, but didn't reach end of line around data row: 2187736. Parsing extra columns and widening final columnset
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:579
┌ Warning: thread = 1 warning: only found 14 / 15 columns around data row: 29539. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 8 warning: parsed expected 15 columns, but didn't reach end of line around data row: 726030. Parsing extra columns and widening final columnset
└ @ CSV /home/joelkik/.julia/packages/CSV/

,marketplace,customer_id,review_id,product_id,product_parent,product_title
,String3,Int64,String15,String15,Int64,String
1,US,32158956,R1KKOXHNI8MSXU,B01KL6O72Y,24485154,"Easy Tool Stainless Steel Fruit Pineapple Corer Slicer Peeler Cut (One size, sliver)"
2,US,2714559,R26SP2OPDK4HT7,B01ID3ZS5W,363128556,V28 Women Cowl Neck Knit Stretchable Elasticity Long Sleeve Slim Fit Sweater Dress
3,US,12608825,RWQEDYAX373I1,B01I497BGY,811958549,James Fiallo Men's 12-Pairs Low Cut Athletic Sport Socks
4,US,25482800,R231YI7R4GPF6J,B01HDXFZK6,692205728,Belfry Gangster 100% Wool Stain-Resistant Crushable Dress Fedora in 4 Colors
5,US,9310286,R3KO3W45DD0L1K,B01G6MBEBY,431150422,JAEDEN Women's Beaded Spaghetti Straps Sexy Long Formal Prom Evening Dresses
6,US,26631939,R1C4QH63NFL5NJ,B01FWRXN0Y,366144407,Levi's Boys' 514 Straight Fit Jeans
7,US,48785098,R2GP65O1U9N7BP,B01EXNH1HE,786052021,Minimalist Wallet & Credit Card Holder Men with Slim Design by Raw
8,US,39548589,R3O29CT5MQQ3XQ,B01E7OL09O,108920964,Harriton Men's Barbados Textured Camp Shirt
9,US,29355866,R1ZECD2AA8QFF6,B01DXHX81O,317132458,Jockey Women's Underwear Supersoft Brief - 3 Pack


In [4]:

function calTermAndDocTerm(docs)
    println("ThreadId ",Threads.threadid())
    termDict=Dict()
    docTerm=Dict()
    docTermCount=[]
    for i in 1:length(docs)
        dt=Dict()
        for t in docs[i].tokens
            if haskey(termDict, t)
                termDict[t]=termDict[t]+1
            else
                termDict[t]=1
            end
            if haskey(docTerm, t)
                push!(docTerm[t], i)
            else
                docTerm[t]=Set(i)
            end
            if haskey(dt, t)
                dt[t]=dt[t]+1
            else
                dt[t]=1
            end
        end
        push!(docTermCount, dt)
    end
    return [docTerm, docTermCount]
end

function preprocessDocsGetDocTerm(st, ed)
    docs=[]
    for i in st:ed
        if typeof(df.review_body[i])==String
            sd=TokenDocument(df.review_body[i])
            prepare!(sd, strip_stopwords)
            prepare!(sd, strip_non_letters)
            push!(docs, sd)
        end
    end
    return calTermAndDocTerm(docs)
end


preprocessDocsGetDocTerm (generic function with 1 method)

In [5]:
totalDocs=nrow(df)
n = floor(Int, totalDocs/8)
dT1 = Threads.@spawn preprocessDocsGetDocTerm(1, n)
#println("1")
dT2 = Threads.@spawn preprocessDocsGetDocTerm(n+1, n*2)
#println("2")
dT3 = Threads.@spawn preprocessDocsGetDocTerm(n*2+1, n*3)
#println("3")
dT4 = Threads.@spawn preprocessDocsGetDocTerm(n*3+1, n*4)
dT5 = Threads.@spawn preprocessDocsGetDocTerm(n*4+1, n*5)
dT6 = Threads.@spawn preprocessDocsGetDocTerm(n*5+1, n*6)
dT7 = Threads.@spawn preprocessDocsGetDocTerm(n*6+1, n*7)
dT8 = Threads.@spawn preprocessDocsGetDocTerm(n*7+1, totalDocs)
#println("4")
wait(dT1)
wait(dT2)
wait(dT3)
wait(dT4)
wait(dT5)
wait(dT6)
wait(dT7)
wait(dT8)


ThreadId 5
ThreadId 1
ThreadId 6
ThreadId 2
ThreadId 7
ThreadId 8
ThreadId 4
ThreadId 3


In [6]:
docTerms=Dict()
function mergeDocTerm(d)
    for (k,v) in d
        if haskey(docTerms, k)
            docTerms[k]+=length(v)
        else
            docTerms[k]=length(v)
        end
    end
end

res1 = fetch(dT1)
res2 = fetch(dT2)
res3 = fetch(dT3)
res4 = fetch(dT4)
res5 = fetch(dT5)
res6 = fetch(dT6)
res7 = fetch(dT7)
res8 = fetch(dT8)

mergeDocTerm(res1[1])
mergeDocTerm(res2[1])
mergeDocTerm(res3[1])
mergeDocTerm(res4[1])
mergeDocTerm(res5[1])
mergeDocTerm(res6[1])
mergeDocTerm(res7[1])
mergeDocTerm(res8[1])

In [7]:
IDF=Dict()
for (k,v) in docTerms
    IDF[k]=log2(totalDocs/length(v))
end

In [8]:
function calTfIdf(docTermCount)
    tf_idf=[]
    for i in 1:length(docTermCount)
        dt=Dict()
        for (k,v) in docTermCount[i]
            dt[k]=v*IDF[k]
        end
        push!(tf_idf, dt)
    end
    return tf_idf
end

calTfIdf (generic function with 1 method)

In [9]:
tfIdf1 = Threads.@spawn calTfIdf(res1[2])
tfIdf2 = Threads.@spawn calTfIdf(res2[2])
tfIdf3 = Threads.@spawn calTfIdf(res3[2])
tfIdf4 = Threads.@spawn calTfIdf(res4[2])
tfIdf5 = Threads.@spawn calTfIdf(res5[2])
tfIdf6 = Threads.@spawn calTfIdf(res6[2])
tfIdf7 = Threads.@spawn calTfIdf(res7[2])
tfIdf8 = Threads.@spawn calTfIdf(res8[2])
wait(tfIdf1)
wait(tfIdf2)
wait(tfIdf3)
wait(tfIdf4)
wait(tfIdf5)
wait(tfIdf6)
wait(tfIdf7)
wait(tfIdf8)

In [10]:
endTime = Dates.Time(Dates.now()) 

13:47:28.974

In [11]:
println("Time Taken: ", endTime-startTime)

Time Taken: 659056000000 nanoseconds


In [ ]:
#print(fetch(tfIdf1))